# MOID

[Minimum orbit intersection distance (MOID)](https://en.wikipedia.org/wiki/Minimum_orbit_intersection_distance)

In [35]:
# Import standard modules
import sqlite3
from astropy.time import Time
from astropy import units as u
from scipy.spatial.distance import cdist

import spiceypy as sp
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore', append=True)

AU2KM = sp.convrt(1, "au", "km")
KM2AU = sp.convrt(1, "km", "au")
# RAD2DEG = sp.dpr()
# SEC2YR = sp.convrt(1, "seconds", "years")

# Load the SPICE kernel meta file
sp.furnsh('kernel_meta.txt')

# Extract the GM value of the Sun
GM_SUN = sp.bodvcd(bodyid=10, item='GM', maxn=1)[1][0]

# Set times
T1_UTC = Time("2020-05-20", format='iso', scale='utc')
T2_UTC = Time("2020-06-10", format='iso', scale='utc')
times_utc = np.arange(T1_UTC, T2_UTC, 1*u.h)
times_et = [sp.utc2et(t.strftime("%Y-%m-%dT%H:%M:%S")) for t in times_utc]

# Extract orbit data of the comet C/2019 Y4 (ATLAS)
con = sqlite3.connect('../_databases/_comets/mpc_comets.db')
elt_y4 = pd.read_sql("""
SELECT PERIHELION_AU, ECCENTRICITY, INCLINATION_DEG, LONG_OF_ASC_NODE_DEG, ARG_OF_PERIH_DEG,
MEAN_ANOMALY_DEG, EPOCH_ET FROM comets_main WHERE NAME="C/2019 Y4 (ATLAS)"
""", con)
elt_y4 = [
    elt_y4['PERIHELION_AU'].values[0]*AU2KM,
    elt_y4['ECCENTRICITY'].values[0],
    np.deg2rad(elt_y4['INCLINATION_DEG'].values[0]),
    np.deg2rad(elt_y4['LONG_OF_ASC_NODE_DEG'].values[0]),
    np.deg2rad(elt_y4['ARG_OF_PERIH_DEG'].values[0]),
    np.deg2rad(elt_y4['MEAN_ANOMALY_DEG'].values[0]),
    elt_y4['EPOCH_ET'].values[0],
    GM_SUN
]

In [38]:
# Comet (not in the ephemeris)
pos_y4 = np.array([sp.conics(elt_y4, _t)[:3] for _t in times_et])
# Solar Orbiter (in the ephemeris)
pos_so = np.array([sp.spkgeo(targ=-144, et=_t, ref='ECLIPJ2000', obs=10)[0][:3] for _t in times_et])

min_dist_y4 = np.min(np.linalg.norm(pos_y4, axis=1))
min_dist_so = np.min(np.linalg.norm(pos_so, axis=1))
moid_y4_so = cdist(pos_y4, pos_so).min()
print(f"Minimum distance between the comet and Solar Orbiter: {min_dist_y4*KM2AU:.3f} au")
print(f"Minimum distance between Solar Orbiter and the comet: {min_dist_so*KM2AU:.3f} au")
print(f"MOID between the comet and Solar Orbiter: {moid_y4_so*KM2AU:.3f} au")

Minimum distance between the comet and Solar Orbiter: 0.253 au
Minimum distance between Solar Orbiter and the comet: 0.521 au
MOID between the comet and Solar Orbiter: 0.270 au
